In [0]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import IsolationForest

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost-1.csv', name='media_prediction_and_its_cost-1.csv', size=16667988, modificationTime=1682550968000),
 FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost-2.csv', name='media_prediction_and_its_cost-2.csv', size=16667988, modificationTime=1682551383000),
 FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost.csv', name='media_prediction_and_its_cost.csv', size=16667988, modificationTime=1682548452000),
 FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost_NEW.csv', name='media_prediction_and_its_cost_NEW.csv', size=14592433, modificationTime=1682555895000)]

In [0]:
# path = 'D:/Study/McGill University - MMA/Courses/INSY 695/Team Project/Data/'
# df_raw = pd.read_csv(path+"media prediction and its cost NEW.csv")

# File location and type
file_location = "/FileStore/tables/media_prediction_and_its_cost_NEW.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df_raw = df.toPandas()

In [0]:
num_cols, cate_cols, pre_dum_cols = [], [], []
for c in df_raw.columns:
    uni_len = len(set(df_raw[c]))
    dtype = df_raw[c].dtypes

    if dtype == object:
        cate_cols.append(c)
    else: # dtype == float
        if uni_len == 2:
            pre_dum_cols.append(c)
        elif uni_len < 10:
            num_cols.append(c)
            print("Number of unique values: %d \t Datatype: %s \t Column name: %s" %(uni_len, dtype, c))
        else:
            num_cols.append(c)

Number of unique values: 6 	 Datatype: int32 	 Column name: unit_sales(in millions)
Number of unique values: 6 	 Datatype: int32 	 Column name: total_children
Number of unique values: 5 	 Datatype: int32 	 Column name: avg_cars_at home(approx)15
Number of unique values: 6 	 Datatype: int32 	 Column name: num_children_at_home
Number of unique values: 5 	 Datatype: int32 	 Column name: avg_cars_at home(approx)18


In [0]:
df_raw.head()

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,...,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4,Bag Stuffers,USA,M,F,...,8670,5415,13610,1,1,1,1,1,"Daily Paper, Radio",126.62
1,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4,Bag Stuffers,USA,M,F,...,18670,5415,3610,1,1,1,1,1,"Daily Paper, Radio",126.62
2,Breakfast Foods,Frozen Foods,Food,5.52,2.5944,3,Cash Register Lottery,USA,M,M,...,18670,5415,3610,1,1,1,1,1,"Daily Paper, Radio",59.86
3,Breakfast Foods,Frozen Foods,Food,3.68,1.3616,2,High Roller Savings,USA,S,F,...,18670,5415,3610,1,1,1,1,1,"Daily Paper, Radio",84.16
4,Breakfast Foods,Frozen Foods,Food,3.68,1.1776,2,Cash Register Lottery,USA,M,F,...,18670,5415,3610,1,1,1,1,1,In-Store Coupon,95.78


# Model Development

In [0]:
y = df_raw['cost']
X = df_raw.drop(columns = ['cost'])
X = pd.get_dummies(X)

In [0]:
from sklearn.model_selection import train_test_split
X_train_final, X_test_final, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [0]:
X_test_final, X_val_final, y_test, y_val = train_test_split(X_test_final, y_test, test_size = 0.5, random_state = 0)

# TPOT

In [0]:
! pip install tpot

     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |▌                               | 3.3 MB 78.0 MB/s eta 0:00:03

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 200.3 MB 10 kB/s 
     |████████████████████████████████| 139 kB 82.0 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=bfbaf27cefa88de75487283fedbd433483122ecaae63093f52765a591539e0fc
  Stored in directory: /root/.cache/pip/wheels/48/8c/93/3afb1916772591fe6bcc25cdf8b1c5bdc362f0ec8e2f0fd413
Successfully built stopit
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e7b9e488-edb8-4e73-b495-a84a207db0db/bin/python -m pip install --upgrade pip' command.


In [0]:
from tpot import TPOTRegressor
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=0)
tpot_model = tpot.fit(X_train_final, y_train)
y_val_pred_tpot = tpot_model.predict(X_val_final)

---------------------------------------------------------------------------
The Python process exited with exit code 137 (SIGKILL: Killed). This may have been caused by an OOM error. Check your command's memory usage.

---------------------------------------------------------------------------
The last 10 KB of the process's stderr and stdout can be found below. See driver logs for full logs.
---------------------------------------------------------------------------
Last messages on stderr:
Thu Apr 27 00:51:46 2023 Connection to spark from PID  914
Thu Apr 27 00:51:46 2023 Initialized gateway on port 39435
Thu Apr 27 00:51:46 2023 Connected to spark.
/local_disk0/.ephemeral_nfs/envs/pythonEnv-e7b9e488-edb8-4e73-b495-a84a207db0db/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models

In [0]:
from sklearn.metrics import mean_squared_error
tpot_pred_mse = mean_squared_error(y_val, y_val_pred_tpot, squared=False)
tpot_pred_mse

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
import mlflow
from mlflow.models import make_metric
import os
import matplotlib.pyplot as plt

# creating the evaluation dataframe
eval_data = X_val_final.copy()
eval_data["target"] = y_test


def squared_diff_plus_one(eval_df, _builtin_metrics):
    """
    This example custom metric function creates a metric based on the ``prediction`` and
    ``target`` columns in ``eval_df``.
    """
    return np.sum(np.abs(eval_df["prediction"] - eval_df["target"] + 1) ** 2)


def sum_on_target_divided_by_two(_eval_df, builtin_metrics):
    """
    This example custom metric function creates a metric derived from existing metrics in
    ``builtin_metrics``.
    """
    return builtin_metrics["sum_on_target"] / 2


def prediction_target_scatter(eval_df, _builtin_metrics, artifacts_dir):
    """
    This example custom artifact generates and saves a scatter plot to ``artifacts_dir`` that
    visualizes the relationship between the predictions and targets for the given model to a
    file as an image artifact.
    """
    plt.scatter(eval_df["prediction"], eval_df["target"])
    plt.xlabel("Targets")
    plt.ylabel("Predictions")
    plt.title("Targets vs. Predictions")
    plot_path = os.path.join(artifacts_dir, "example_scatter_plot.png")
    plt.savefig(plot_path)
    return {"example_scatter_plot_artifact": plot_path}


experiment_id = "4410962850347378"

with mlflow.start_run(experiment_id = experiment_id) as run:
    mlflow.sklearn.log_model(tpot_model, "model")

    model_uri = mlflow.get_artifact_uri("model")
    
    result = mlflow.evaluate(
        model = model_uri,
        data = eval_data,
        targets = "target",
        model_type = "regressor",
        evaluators=["default"],
        custom_metrics=[
            make_metric(
                eval_fn=squared_diff_plus_one,
                greater_is_better=False,
            ),
            make_metric(
                eval_fn=sum_on_target_divided_by_two,
                greater_is_better=True,
            ),
        ],
        custom_artifacts=[prediction_target_scatter],
    )

print(f"metrics:\n{result.metrics}")
print(f"artifacts:\n{result.artifacts}")
